In [ ]:
print("comecou script")
#importa a biblioteca para manipulação de objetos JSON
import json
#importa a biblioteca para manipulação de bancos de dados sqlite
import sqlite3
#importa a biblioteca para manipulação de arquivos xlsx do office
import xlsxwriter
# Conecta/cria o banco de dados para os dados de tedio e estresse
dados = sqlite3.connect("C:\Python\dados\dados.sqlite")
#cria os cursores para manipulação dos dados de tedio e estresse
tedio = dados.cursor()
estresse = dados.cursor()
# exclui as tabelas de tedio e estresse caso existam
tedio.execute("""DROP TABLE tedio """)
estresse.execute("""DROP TABLE estresse """)
# cria/recria as tabelas de tedio e estresse
tedio.execute("""CREATE TABLE tedio (game_id int, game_name text, uuid int, timestamp int, angry float, disgusted float, fear float, sad float, surprised float, happy float, p1 float, p2 float, p3 float, p4 float, p5 float, p6 float, p7 float, p8 float, p9 float, p10 float, p11 float, p12 float, p13 float, p14 float, p15 float, p16 float, p17 float, p18 float, p19 float, p20 float, p21 float, p22 float, p23 float, p24 float)""")
estresse.execute("""CREATE TABLE estresse (game_id int, game_name text, uuid int, timestamp int, angry float, disgusted float, fear float, sad float, surprised float, happy float, p1 float, p2 float, p3 float, p4 float, p5 float, p6 float, p7 float, p8 float, p9 float, p10 float, p11 float, p12 float, p13 float, p14 float, p15 float, p16 float, p17 float, p18 float, p19 float, p20 float, p21 float, p22 float, p23 float, p24 float)""")
# apaga o conteúdo das tabelas tedio e estresse cada execução
tedio.execute("""DELETE FROM tedio """)
estresse.execute("""DELETE FROM estresse """)
#conecta ao banco de dados do estudo
estudo = sqlite3.connect('C:\Python\dados\database.sqlite')
#cria cursor para manipulação de dados da tabela games
jogos = estudo.cursor()
# cria a lista de jogos validos
jogos.execute("SELECT id, name FROM games order by cast(id as int)")
interacoes1 = 0
interacoes2 = 0
# varre a lista de jogos validos para selecionar os dados capturados de cada jogador/jogo
print("comecou coletar")
for g1 in jogos:
    jogadores = estudo.cursor()
    jogadores.execute("SELECT distinct uuid, fk_game FROM logs where fk_game = " + str(g1[0]) + " order by cast(uuid as int)")
    for p1 in jogadores:
        emocoes = estudo.cursor()
        emocoes.execute("SELECT count(*), data FROM logs where uuid = " + str(p1[0]) + " and fk_game = " + str(p1[1]) + " and data like '%emotion%' order by cast(uuid as int)")
        qtd_capturas = emocoes.fetchone()[0]
        # se o jogo/jogador não tiver pelo menos 50 emocões capturadas os dados desse jogo/jogador não são considerados
        if qtd_capturas >= 50 :
           interacoes1 = 0 
           emocoes.execute("SELECT distinct uuid, fk_game, data FROM logs where uuid = " + str(p1[0]) + " and fk_game = " + str(p1[1]) + " and data like '%emotion%' order by timestamp")
           for e1 in emocoes:
               interacoes1 = interacoes1 + 1
               #ignora-se as primeiras 9 capturas do jogo/jogador e separa e insere na tabela tedio as 20 seguintes
               if interacoes1 > 9 and interacoes1 < 30:
                  #cria um array de objetos JSON
                  vdados = json.loads(e1[2])
                  #cria um list para guardar os dados de cada elemento dos objetos json sensiveis para o resultado
                  vValues = []
                  #interaçao para percorrer todos os objetos json que necessitamos pegar o valor - Alterado para pegar apenas a primeira e ultima interação 
                  emocao = 0
                  for vdado in vdados:
                      emocao += 1
                      if emocao == 1 or emocao == len(vdados):
                         #print("emocao: " + str(emocao))
                         v1 = json.loads(vdado.get('d'))
                         if not isinstance(v1, str) :
                            v2 = v1.get('e')
                            if v2:
                               vValues.append(vdado.get('t'))
                               for vn in v2:
                                   vValues.append(vn.get('value'))
                               v3 = v1.get('p')
                               if v3:
                                  for vn1 in v3:
                                      vValues.append(vn1)
                               if len(vValues) == 31:
                                  tedio.execute("insert into tedio(game_id , game_name , uuid , timestamp , angry , disgusted , fear , sad , surprised , happy, p1 , p2 , p3 , p4 , p5 , p6 , p7 , p8 , p9 , p10 , p11 , p12 , p13 , p14 , p15 , p16 , p17 , p18 , p19 , p20 , p21 , p22 , p23 , p24 ) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",(e1[1] ,g1[1],e1[0],vValues[0],vValues[1],vValues[2],vValues[3],vValues[4],vValues[5],vValues[6],vValues[7],vValues[8],vValues[9],vValues[10],vValues[11],vValues[12],vValues[13],vValues[14],vValues[15],vValues[16],vValues[17],vValues[18],vValues[19],vValues[20],vValues[21],vValues[22],vValues[23],vValues[24],vValues[25],vValues[26],vValues[27],vValues[28],vValues[29],vValues[30])) 
                               vValues = []
                             
          
          
           emocoes2 = estudo.cursor()
           emocoes2.execute("SELECT distinct uuid, fk_game, data FROM logs where uuid = " + str(p1[0]) + " and fk_game = " + str(p1[1]) + " and data like '%emotion%' order by timestamp desc")          
           interacoes2 = 0
           for e2 in emocoes2:
               interacoes2 = interacoes2 + 1
               #insere-se as 20 ultimas capturas do jogo/jogador na tabela estresse
               if interacoes2 < 21 :
                   #cria um array de objetos JSON
                   vdados2 = json.loads(e2[2])
                   #cria um list para guardar os dados de cada elemento dos objetos json sensiveis para o resultado 
                   vValues2 = []
                   #interaçao para percorrer todos os objetos json que necessitamos pegar o valor
                   emocao2 = 0
                   for vdado2 in vdados2:
                      emocao2 += 1
                      if emocao2 == 1 or emocao2 == len(vdados2):
                         #print("emocao2: " + str(emocao2))   
                         v21 = json.loads(vdado2.get('d'))
                         if not isinstance(v21, str) :
                            v22 = v21.get('e')
                            if v22:
                               vValues2.append(vdado2.get('t'))
                               for vn2 in v22:
                                   vValues2.append(vn2.get('value'))
                               v23 = v21.get('p')
                               if v23:
                                  for vn21 in v23:
                                      vValues2.append(vn21)
                               if len(vValues2) == 31:
                                  estresse.execute("insert into estresse(game_id , game_name , uuid , timestamp , angry , disgusted , fear , sad , surprised , happy, p1 , p2 , p3 , p4 , p5 , p6 , p7 , p8 , p9 , p10 , p11 , p12 , p13 , p14 , p15 , p16 , p17 , p18 , p19 , p20 , p21 , p22 , p23 , p24 ) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",(e2[1] ,g1[1],e2[0],vValues2[0],vValues2[1],vValues2[2],vValues2[3],vValues2[4],vValues2[5],vValues2[6],vValues2[7],vValues2[8],vValues2[9],vValues2[10],vValues2[11],vValues2[12],vValues2[13],vValues2[14],vValues2[15],vValues2[16],vValues2[17],vValues2[18],vValues2[19],vValues2[20],vValues2[21],vValues2[22],vValues2[23],vValues2[24],vValues2[25],vValues2[26],vValues2[27],vValues2[28],vValues2[29],vValues2[30])) 
                               vValues2 = []
                   
                                
#faz o commit de todos dados inseridos nas tabelas tedio e estresse
dados.commit()
print("planilha tedio")
#instancia a biblioteca xlsxwriter
from xlsxwriter.workbook import Workbook
#gera um arquivo xlsx
workbook = Workbook('c:\python\dados\dadosTedioC.xlsx')
worksheet = workbook.add_worksheet()
#seleciona todos os dados da tabela tedio
mysel=tedio.execute("select * from tedio")
#cria os titulos das celulas
worksheet.write(0, 0, "game_id")
worksheet.write(0, 1, "game_name")
worksheet.write(0, 2, "uuid")
worksheet.write(0, 3, "timestamp")
worksheet.write(0, 4, "angry")
worksheet.write(0, 5, "disgusted")
worksheet.write(0, 6, "fear")
worksheet.write(0, 7, "sad")
worksheet.write(0, 8, "surprised")
worksheet.write(0, 9, "happy")
worksheet.write(0, 10, "p1")
worksheet.write(0, 11, "p2")
worksheet.write(0, 12, "p3")
worksheet.write(0, 13, "p4")
worksheet.write(0, 14, "p5")
worksheet.write(0, 15, "p6")
worksheet.write(0, 16, "p7")
worksheet.write(0, 17, "p8")
worksheet.write(0, 18, "p9")
worksheet.write(0, 19, "p10")
worksheet.write(0, 20, "p11")
worksheet.write(0, 21, "p12")
worksheet.write(0, 22, "p13")
worksheet.write(0, 23, "p14")
worksheet.write(0, 24, "p15")
worksheet.write(0, 25, "p16")
worksheet.write(0, 26, "p17")
worksheet.write(0, 27, "p18")
worksheet.write(0, 28, "p19")
worksheet.write(0, 29, "p20")
worksheet.write(0, 30, "p21")
worksheet.write(0, 31, "p22")
worksheet.write(0, 32, "p23")
worksheet.write(0, 33, "p24")

#preenche a planilha com dados selecionados da tabela tedio
for i, row in enumerate(mysel):
    worksheet.write(i + 1, 0, row[0])
    worksheet.write(i + 1, 1, row[1])
    worksheet.write(i + 1, 2, row[2])
    worksheet.write(i + 1, 3, row[3])
    worksheet.write(i + 1, 4, row[4])
    worksheet.write(i + 1, 5, row[5])
    worksheet.write(i + 1, 6, row[6])
    worksheet.write(i + 1, 7, row[7])
    worksheet.write(i + 1, 8, row[8])
    worksheet.write(i + 1, 9, row[9])
    worksheet.write(i + 1, 10, row[10])
    worksheet.write(i + 1, 11, row[11])
    worksheet.write(i + 1, 12, row[12])
    worksheet.write(i + 1, 13, row[13])
    worksheet.write(i + 1, 14, row[14])
    worksheet.write(i + 1, 15, row[15])
    worksheet.write(i + 1, 16, row[16])
    worksheet.write(i + 1, 17, row[17])
    worksheet.write(i + 1, 18, row[18])
    worksheet.write(i + 1, 19, row[19])
    worksheet.write(i + 1, 20, row[20])
    worksheet.write(i + 1, 21, row[21])
    worksheet.write(i + 1, 22, row[22])
    worksheet.write(i + 1, 23, row[23])
    worksheet.write(i + 1, 24, row[24])
    worksheet.write(i + 1, 25, row[25])
    worksheet.write(i + 1, 26, row[26])
    worksheet.write(i + 1, 27, row[27])
    worksheet.write(i + 1, 28, row[28])
    worksheet.write(i + 1, 29, row[29])
    worksheet.write(i + 1, 30, row[30])
    worksheet.write(i + 1, 31, row[31])
    worksheet.write(i + 1, 32, row[32])
    worksheet.write(i + 1, 33, row[33])
    
#encerra a instancia da biblioteca xlsxwriter
workbook.close()
print("planilha estresse")
#instancia a biblioteca xlsxwriter
from xlsxwriter.workbook import Workbook
#gera um arquivo xlsx
workbook2 = Workbook('c:\python\dados\dadosEstresseC.xlsx')
worksheet2 = workbook2.add_worksheet()
#seleciona todos os dados da tabela estresse
mysel2=estresse.execute("select * from estresse")
#cria os titulos das celulas
worksheet2.write(0, 0, "game_id")
worksheet2.write(0, 1, "game_name")
worksheet2.write(0, 2, "uuid")
worksheet2.write(0, 3, "timestamp")
worksheet2.write(0, 4, "angry")
worksheet2.write(0, 5, "disgusted")
worksheet2.write(0, 6, "fear")
worksheet2.write(0, 7, "sad")
worksheet2.write(0, 8, "surprised")
worksheet2.write(0, 9, "happy")
worksheet2.write(0, 10, "p1")
worksheet2.write(0, 11, "p2")
worksheet2.write(0, 12, "p3")
worksheet2.write(0, 13, "p4")
worksheet2.write(0, 14, "p5")
worksheet2.write(0, 15, "p6")
worksheet2.write(0, 16, "p7")
worksheet2.write(0, 17, "p8")
worksheet2.write(0, 18, "p9")
worksheet2.write(0, 19, "p10")
worksheet2.write(0, 20, "p11")
worksheet2.write(0, 21, "p12")
worksheet2.write(0, 22, "p13")
worksheet2.write(0, 23, "p14")
worksheet2.write(0, 24, "p15")
worksheet2.write(0, 25, "p16")
worksheet2.write(0, 26, "p17")
worksheet2.write(0, 27, "p18")
worksheet2.write(0, 28, "p19")
worksheet2.write(0, 29, "p20")
worksheet2.write(0, 30, "p21")
worksheet2.write(0, 31, "p22")
worksheet2.write(0, 32, "p23")
worksheet2.write(0, 33, "p24")
#preenche a planilha com dados selecionados da tabela estresse
for i2, row2 in enumerate(mysel2):
    worksheet2.write(i2 + 1, 0, row2[0])
    worksheet2.write(i2 + 1, 1, row2[1])
    worksheet2.write(i2 + 1, 2, row2[2])
    worksheet2.write(i2 + 1, 3, row2[3])
    worksheet2.write(i2 + 1, 4, row2[4])
    worksheet2.write(i2 + 1, 5, row2[5])
    worksheet2.write(i2 + 1, 6, row2[6])
    worksheet2.write(i2 + 1, 7, row2[7])
    worksheet2.write(i2 + 1, 8, row2[8])
    worksheet2.write(i2 + 1, 9, row2[9])
    worksheet2.write(i2 + 1, 10, row2[10])
    worksheet2.write(i2 + 1, 11, row2[11])
    worksheet2.write(i2 + 1, 12, row2[12])
    worksheet2.write(i2 + 1, 13, row2[13])
    worksheet2.write(i2 + 1, 14, row2[14])
    worksheet2.write(i2 + 1, 15, row2[15])
    worksheet2.write(i2 + 1, 16, row2[16])
    worksheet2.write(i2 + 1, 17, row2[17])
    worksheet2.write(i2 + 1, 18, row2[18])
    worksheet2.write(i2 + 1, 19, row2[19])
    worksheet2.write(i2 + 1, 20, row2[20])
    worksheet2.write(i2 + 1, 21, row2[21])
    worksheet2.write(i2 + 1, 22, row2[22])
    worksheet2.write(i2 + 1, 23, row2[23])
    worksheet2.write(i2 + 1, 24, row2[24])
    worksheet2.write(i2 + 1, 25, row2[25])
    worksheet2.write(i2 + 1, 26, row2[26])
    worksheet2.write(i2 + 1, 27, row2[27])
    worksheet2.write(i2 + 1, 28, row2[28])
    worksheet2.write(i2 + 1, 29, row2[29])
    worksheet2.write(i2 + 1, 30, row2[30])
    worksheet2.write(i2 + 1, 31, row2[31])
    worksheet2.write(i2 + 1, 32, row2[32])
    worksheet2.write(i2 + 1, 33, row2[33])
   
#encerra a instancia da biblioteca xlsxwriter    
workbook2.close()
print("Dados Separados com sucesso!")

comecou script
comecou coletar
emocao: 1
emocao: 16
emocao: 1
emocao: 10
emocao: 1
emocao: 16
emocao: 1
emocao: 11
emocao: 1
emocao: 12
emocao: 1
emocao: 17
emocao: 1
emocao: 18
emocao: 1
emocao: 21
emocao: 1
emocao: 20
emocao: 1
emocao: 19
emocao: 1
emocao: 19
emocao: 1
emocao: 21
emocao: 1
emocao: 22
emocao: 1
emocao: 15
emocao: 1
emocao: 20
emocao: 1
emocao: 23
emocao: 1
emocao: 21
emocao: 1
emocao: 22
emocao: 1
emocao: 18
emocao: 1
emocao: 24
emocao2: 1
emocao2: 24
emocao2: 1
emocao2: 14
emocao2: 1
emocao2: 30
emocao2: 1
emocao2: 30
emocao2: 1
emocao2: 24
emocao2: 1
emocao2: 26
emocao2: 1
emocao2: 27
emocao2: 1
emocao2: 26
emocao2: 1
emocao2: 22
emocao2: 1
emocao2: 22
emocao2: 1
emocao2: 24
emocao2: 1
emocao2: 22
emocao2: 1
emocao2: 27
emocao2: 1
emocao2: 22
emocao2: 1
emocao2: 22
emocao2: 1
emocao2: 15
emocao2: 1
emocao2: 24
emocao2: 1
emocao2: 24
emocao2: 1
emocao2: 22
emocao2: 1
emocao2: 26
emocao: 1
emocao: 15
emocao: 1
emocao: 9
emocao: 1
emocao: 9
emocao: 1
emocao: 14
emocao: